In [1]:
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
from torchvision.models import resnet18
import pickle
import os

In [2]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4), #32x32 -> 40x40 -> 32x32
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(), #PIL to tensor
    transforms.Normalize(#apprently help to converge.
        mean=[0.5071, 0.4867, 0.4408], 
        std=[0.2675, 0.2565, 0.2761]
    ),
])#help the model to be more robust

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5071, 0.4867, 0.4408],
        std=[0.2675, 0.2565, 0.2761] #necessary to have the same kind of data for evaluation
    ),
])

In [3]:

trainset = torchvision.datasets.CIFAR100(
    root='./data',
    train=True,
    download=True,
    transform=transform_train
)

testset = torchvision.datasets.CIFAR100(
    root='./data',
    train=False,
    download=True,
    transform=transform_test
)

trainloader = DataLoader(trainset, batch_size=256, shuffle=True, num_workers=8)
testloader = DataLoader(testset, batch_size=256, shuffle=False, num_workers=8)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
model = resnet18(weights=None)  # pas ImageNet pretrained

#H_out = (H + 2p − k) / s + 1
#Donc sans toucher H_out = (32 + 2x3 - 7) / 2  + 1 = 16 
#Puis avec le max pool on tombe à 8. 
# Très génant car les canaux stocknte des motifs mais pas la position, donc en passant de 32 à 8 on "flouterai" enormement l'image

# Modifie la première couche pour prendre du 32x32
model.conv1 = nn.Conv2d(
    3, 64, kernel_size=3, stride=1, padding=1, bias=False
)
model.maxpool = nn.Identity() #out = 64x32x32 bien mieux que 64x8x8

# Modifier la dernière couche pour avoir 100 classes
model.fc = nn.Linear(512, 100)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")
model = model.to(device)

Device: cuda


In [5]:
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(
    model.parameters(),
    lr=0.1,
    momentum=0.9,
    weight_decay=5e-4
)

scheduler = torch.optim.lr_scheduler.MultiStepLR(
    optimizer,
    milestones=[100, 150],
    gamma=0.1
)

In [6]:
def train_one_epoch(model, loader):
    model.train()
    correct, total, running_loss = 0, 0, 0

    for inputs, targets in loader:
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    acc = 100. * correct / total
    return running_loss / len(loader), acc


In [7]:
@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    correct, total = 0, 0

    for inputs, targets in loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    return 100. * correct / total

In [8]:
checkpoint = torch.load("checkpoints/checkpoint_epoch_20.pth")

model.load_state_dict(checkpoint["model_state"])
optimizer.load_state_dict(checkpoint["optimizer_state"])
scheduler.load_state_dict(checkpoint["scheduler_state"])

start_epoch = checkpoint["epoch"]
model = model.to(device)

C:\Users\mathi\AppData\Local\Temp\ipykernel_28876\3999325711.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("checkpoints/checkpoint_epoch_20.pth

In [9]:
model = model.to(device)

train_losses = checkpoint["train_losses"]
train_accs = checkpoint["train_accs"]
test_accs = checkpoint["test_accs"]

In [ ]:
train_accs, test_accs, train_losses = [], [], []

In [ ]:
num_epochs = 200

save_every = 10
os.makedirs("checkpoints", exist_ok=True)
torch.backends.cudnn.benchmark = True
for epoch in range(start_epoch, num_epochs):
    train_loss, train_acc = train_one_epoch(model, trainloader)
    test_acc = evaluate(model, testloader)
    scheduler.step()

    train_accs.append(train_acc)
    test_accs.append(test_acc)
    train_losses.append(train_loss)
    #sauvegarde périodique
    if (epoch + 1) % save_every == 0:
        checkpoint = {
            "epoch": epoch + 1,
            "model_state": model.state_dict(),
            "optimizer_state": optimizer.state_dict(),
            "scheduler_state": scheduler.state_dict(),
            "train_losses": train_losses,
            "train_accs": train_accs,
            "test_accs": test_accs,
        }
        torch.save(checkpoint, f"checkpoints/checkpoint_epoch_{epoch+1}.pth")

    print(f"Epoch {epoch:03d} | Train Acc: {train_acc:.2f}% | Test Acc: {test_acc:.2f}%")

torch.save(model.state_dict(), "resnet18_cifar100.pth")

history = {
    "train_losses": train_losses,
    "train_accs": train_accs,
    "test_accs": test_accs
}

with open("training_history.pkl", "wb") as f:
    pickle.dump(history, f)

Epoch 020 | Train Acc: 77.33% | Test Acc: 68.08%
Epoch 021 | Train Acc: 79.36% | Test Acc: 64.39%
Epoch 022 | Train Acc: 79.46% | Test Acc: 61.94%
Epoch 023 | Train Acc: 78.97% | Test Acc: 62.64%
Epoch 024 | Train Acc: 78.63% | Test Acc: 61.40%
Epoch 025 | Train Acc: 78.85% | Test Acc: 61.52%
Epoch 026 | Train Acc: 78.80% | Test Acc: 59.35%
Epoch 027 | Train Acc: 79.06% | Test Acc: 59.70%
Epoch 028 | Train Acc: 79.38% | Test Acc: 60.12%
Epoch 029 | Train Acc: 79.92% | Test Acc: 61.94%
Epoch 030 | Train Acc: 79.71% | Test Acc: 62.68%
